## Primer on Python Decorators

Reference:

- https://realpython.com/primer-on-python-decorators/
- https://github.com/realpython/materials/tree/master/primer-on-python-decorators

By **definition**, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

## First-Class Objects

In Python, functions are **first-class objects**. This means that *functions can be passed around and used as arguments*, just like any other object (string, int, float, list, and so on). Consider the following three functions:

In [1]:
def say_hello(name):
    print(f"Hello {name}")

def be_awesome(name):
    print(f"Yo {name}, together we are the awesomest!")

def greet_bob(greeter_func):
    return(greeter_func("Bob"))

say_hello("Bob")
be_awesome("Bob")
greet_bob(say_hello)
greet_bob(be_awesome)

Hello Bob
Yo Bob, together we are the awesomest!
Hello Bob
Yo Bob, together we are the awesomest!


## Inner Functions

It’s possible to define functions inside other functions. Such functions are called inner functions. Here’s an example of a function with two inner functions:

In [2]:
def parent():
    print("Printing from the parent() function")
    
    def first_child():
        print("Printing from the first_child() function")
        
    def second_child():
        print("printing from the second_child() function")
    
    first_child()
    second_child()

In [3]:
parent()

Printing from the parent() function
Printing from the first_child() function
printing from the second_child() function


## Returning Functions From Functions

Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer parent() function:

In [4]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

**Note that you are returning first_child without the parentheses. Recall that this means that you are returning a reference to the function first_child. In contrast first_child() with parentheses refers to the result of evaluating the function**. This can be seen in the following example:

In [5]:
first = parent(1)
second = parent(2)
print(first())
print(second())

print(first)
print(second)

Hi, I am Emma
Call me Liam
<function parent.<locals>.first_child at 0x7fe7a0a105e0>
<function parent.<locals>.second_child at 0x7fe7a0a10700>


## Simple Decorators

Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator. Let’s start with an example:

In [6]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happending after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")
    

say_whee = my_decorator(say_whee)

In [7]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happending after the function is called.


In [8]:
@my_decorator
def say_whee():
    print("Whee!")

In [9]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happending after the function is called.


In [10]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

This is so-called **decoration**. In effect, the name say_whee now points to the wrapper() inner function. Remember that you return wrapper as a function when you call my_decorator(say_whee) *<function __main__.my_decorator.<locals>.wrapper()>*. The wrapper() has a reference to the original say_whee() as func, and calls that function between the two calls to print().
    
Put simply: **decorators wrap a function, modifying its behavior**.

Before moving on, let’s have a look at a second example. Because wrapper() is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [11]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper


def say_whee():
    print("Whee!")

# change the time when the code can be executed
say_whee = not_during_the_night(say_whee)

say_whee()

In [12]:
@not_during_the_night
def say_whee():
    print("Whee!")

say_whee()

## Reusing Decorators

Recall that a decorator is just a regular Python function. All the usual tools for easy reusability are available. Let’s move the decorator to its own module that can be used in many other functions.

In [13]:
from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")

In [14]:
say_whee()

Whee!
Whee!


## Decorating Functions With Arguments

The practice is to use *args and **kwargs in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments.

In [15]:
from decorators import do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

In [16]:
greet("World")

Hello World
Hello World


## Returning Values From Decorated Functions

Make sure the wrapper function returns the return value of the decorated function.

In [17]:
from decorators import do_twice

@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [18]:
hi_adam = return_greeting("Adam")
print(hi_adam)

Creating greeting
Creating greeting
Hi Adam


In [19]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
        
    return wrapper_do_twice

@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

hi_adam = return_greeting("Adam")
print(hi_adam)

Creating greeting
Creating greeting
Hi Adam


## Introspection

 Introspection is the ability of an object to know about its own attributes at runtime. For instance, a function knows its own name and documentation:

In [20]:
print.__name__

'print'

In [21]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [22]:
say_whee

<function decorators.do_twice.<locals>.wrapper_do_twice(*args, **kwargs)>

In [23]:
say_whee.__name__

'wrapper_do_twice'

In [24]:
help(say_whee)

Help on function wrapper_do_twice in module decorators:

wrapper_do_twice(*args, **kwargs)



However, after being decorated, say_whee() has gotten very confused about its identity. It now reports being the wrapper_do_twice() inner function inside the do_twice() decorator. Although technically true, this is not very useful information.

To fix this, decorators should use the **functools.wraps** decorator, which will preserve information about the original function. 

In [28]:
import functools
def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
        
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Whee!")

In [29]:
say_whee

<function __main__.say_whee()>

In [30]:
say_whee.__name__

'say_whee'

In [31]:
help(say_whee)

Help on function say_whee in module __main__:

say_whee()



**Technical Detail**: The @functools.wraps decorator uses the function functools.update_wrapper() to update special attributes like __name__ and __doc__ that are used in the introspection.

## Decorator Template

The formula below is a good boilerplate template for building more complex decorators.

In [32]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator